## Can go to this page: https://www.sec.gov/cgi-bin/srch-edgar

### Use a list of CIK numbers for relevant companies and iterate through 10-K and 10-Q docs

In [1]:
import pandas as pd
import re
import requests
import unicodedata
from bs4 import BeautifulSoup

from helpers import *

## Extract html code

In [2]:
filing = parse_filing(r'https://www.sec.gov/Archives/edgar/data/320193/000032019321000010/0000320193-21-000010.txt', '10-Q')



In [5]:
filing.keys()

dict_keys(['document_sequence', 'document_filename', 'document_description', 'document_code'])

In [7]:
type(filing['document_code'])

bs4.element.Tag

## Identify table of contents elements

In [8]:
table_of_contents = get_table_of_contents(filing['document_code'])
table_of_contents

['Part I',
 'Item 1.',
 'Financial Statements',
 '1',
 'Item 2.',
 'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
 '22',
 'Item 3.',
 'Quantitative and Qualitative Disclosures About Market Risk',
 '29',
 'Item 4.',
 'Controls and Procedures',
 '29',
 'Part II',
 'Item 1.',
 'Legal Proceedings',
 '29',
 'Item 1A.',
 'Risk Factors',
 '29',
 'Item 2.',
 'Unregistered Sales of Equity Securities and Use of Proceeds',
 '30',
 'Item 3.',
 'Defaults Upon Senior Securities',
 '30',
 'Item 4.',
 'Mine Safety Disclosures',
 '30',
 'Item 5.',
 'Other Information',
 '30',
 'Item 6.',
 'Exhibits',
 '31']

In [10]:
## test with tsla
html = 'https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/0001564590-21-004599.txt'
form = '10-K'

In [11]:
tsla_filing = parse_filing(html, form)

In [13]:
tsla_toc = get_table_of_contents(tsla_filing['document_code'])

In [14]:
tsla_toc

['PART I.',
 'Business',
 'Risk Factors',
 'Unresolved Staff Comments',
 'Properties',
 'Legal Proceedings',
 'Mine Safety Disclosures',
 'PART II.',
 'Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities',
 'Selected Consolidated Financial Data',
 'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
 'Quantitative and Qualitative Disclosures About Market Risk',
 'Financial Statements and Supplementary Data',
 'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure',
 'Controls and Procedures',
 'Other Information',
 'PART III.',
 'Directors, Executive Officers and Corporate Governance',
 'Executive Compensation',
 'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters',
 'Certain Relationships and Related Transactions, and Director Independence',
 'Principal Accountant Fees and Services',
 'PART\xa0IV.',
 'Exhibits and Finan

## find all tables

In [ ]:
tables = filing['document_code'].find_all('table', {'style':'border-collapse:collapse;display:inline-table;vertical-align:top;width:100.000%'})
## now we need to identify the correct Table of Contents table, we'll do this later

# trs = tables[3].find_all('tr')
# for tag in trs:
#     tag = trs.find('a', href=True)
#     tag
    
## break table into rows
rows = tables[3].find_all('tr')
# for row in rows:
#     cells = row.find_all(['td', 'th'])
#     for cell in cells:
#         print(cell.name, cell.attrs)
hrefs = rows[5].find('a', href=True).text
hrefs

In [ ]:
hrefs = tables[3].find_all('a', href=True)
for href in hrefs:
    print(href.text)

In [ ]:
filing['document_code'].find(lambda tag:tag.style=='text-align:center' and 'table of contents' in tag.text.lower())


# filing['document_code'].find(lambda tag:tag.name=='b' and 'table of contents' in tag.text.lower())

In [ ]:
filing['document_code'].find(lambda tag:tag.name=='span' and 'table of contents' in tag.text.lower())

In [ ]:
a = filing['document_code'].find_all('div', {"style":"margin-top:6pt;text-align:justify"})

In [ ]:
a[1]

In [ ]:
filing['document_code']